### Exploration

In [1]:
!git clone https://github.com/openai/baselines.git
%cd baselines
!pip install -e .
!pip install stable-baselines
!pip install --upgrade tensorflow==1.14.0
!pip install --upgrade tensorflow-gpu==1.14.0
!pip install gym
!pip install pyarrow
!pip uninstall --yes tensorboard-plugin-wit
%cd ..

fatal: destination path 'baselines' already exists and is not an empty directory.
/project/to-run-for-results/nsw/baselines
Obtaining file:///project/to-run-for-results/nsw/baselines
  Attempting uninstall: baselines
    Found existing installation: baselines 0.1.6
    Uninstalling baselines-0.1.6:
      Successfully uninstalled baselines-0.1.6
  Running setup.py develop for baselines
Requirement already up-to-date: tensorflow==1.14.0 in /opt/conda/lib/python3.7/site-packages (1.14.0)
Requirement already up-to-date: tensorflow-gpu==1.14.0 in /opt/conda/lib/python3.7/site-packages (1.14.0)
/project/to-run-for-results/nsw


In [2]:
import os
import gym
import time
import json
import random
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from datetime import datetime

from stable_baselines.deepq.policies import MlpPolicy
from stable_baselines.common.vec_env.dummy_vec_env import DummyVecEnv

from stable_baselines import deepq
from stable_baselines import bench
from stable_baselines import logger
import tensorflow as tf

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 5255917379908830710,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 9843887573546196686
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 13530770019774154615
 physical_device_desc: "device: XLA_GPU device"]

In [3]:
nslkdd_train = pd.read_feather("/project/datasets/clean-ids-collection/unsw-nb15/clean/designated-train-test-sets/UNSW_NB15_training-set.feather")
nslkdd_test = pd.read_feather("/project/datasets/clean-ids-collection/unsw-nb15/clean/designated-train-test-sets/UNSW_NB15_testing-set.feather")
# data = pd.concat([nslkdd_test], ignore_index=True)
# print(len(data.columns))

nslkdd_train.proto = pd.Categorical(nslkdd_train.proto).codes
nslkdd_train.service = pd.Categorical(nslkdd_train.service).codes
nslkdd_train.state = pd.Categorical(nslkdd_train.state).codes


nslkdd_test.proto = pd.Categorical(nslkdd_test.proto).codes
nslkdd_test.service = pd.Categorical(nslkdd_test.service).codes
nslkdd_test.state = pd.Categorical(nslkdd_test.state).codes


# print(pd.get_dummies(nslkdd_train,columns=['protocol_type' ,'service', 'flag']))
# nslkdd_test = pd.get_dummies(nslkdd_test)
nslkdd_train.head(5)

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,0.000011,117,0,4,2,0,496,0,90909.09375,254,...,1,2,False,0,0,1,2,False,normal,False
1,0.000008,117,0,4,2,0,1762,0,125000.00000,254,...,1,2,False,0,0,1,2,False,normal,False
2,0.000005,117,0,4,2,0,1068,0,200000.00000,254,...,1,3,False,0,0,1,3,False,normal,False
3,0.000006,117,0,4,2,0,900,0,166666.65625,254,...,1,3,False,0,0,2,3,False,normal,False
4,0.000010,117,0,4,2,0,2126,0,100000.00000,254,...,1,3,False,0,0,2,3,False,normal,False


In [4]:
nslkdd_train.info()
nslkdd_train.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53946 entries, 0 to 53945
Data columns (total 44 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   dur                53946 non-null  float32 
 1   proto              53946 non-null  int16   
 2   service            53946 non-null  int8    
 3   state              53946 non-null  int8    
 4   spkts              53946 non-null  int16   
 5   dpkts              53946 non-null  int16   
 6   sbytes             53946 non-null  int32   
 7   dbytes             53946 non-null  int32   
 8   rate               53946 non-null  float32 
 9   sttl               53946 non-null  int16   
 10  dttl               53946 non-null  int16   
 11  sload              53946 non-null  float32 
 12  dload              53946 non-null  float32 
 13  sloss              53946 non-null  int16   
 14  dloss              53946 non-null  int16   
 15  sinpkt             53946 non-null  float32 
 16  dinp

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst
count,53946.000000,53946.00000,53946.000000,53946.000000,53946.000000,53946.000000,5.394600e+04,5.394600e+04,53946.000000,53946.000000,...,53946.000000,53946.000000,53946.000000,53946.000000,53946.000000,53946.000000,53946.000000,53946.000000,53946.000000,53946.000000
mean,1.022188,111.51001,1.514459,3.081934,25.869759,25.967060,1.131112e+04,1.985872e+04,36171.152344,150.903681,...,5.969896,1.035628,3.273533,2.190709,1.486468,3.877711,0.009862,0.160346,3.807437,5.496330
std,3.918624,9.57705,2.442089,0.607792,161.154040,141.707459,2.072230e+05,1.865964e+05,112012.312500,106.750997,...,7.114324,1.084044,4.594301,3.990693,2.516725,6.775809,0.099564,0.561155,4.909204,7.011688
min,0.000000,0.00000,0.000000,0.000000,1.000000,0.000000,2.400000e+01,0.000000e+00,0.000000,0.000000,...,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000
25%,0.004238,111.00000,0.000000,3.000000,4.000000,2.000000,5.320000e+02,1.780000e+02,24.097402,31.000000,...,2.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000
50%,0.388905,111.00000,0.000000,3.000000,10.000000,8.000000,9.000000e+02,3.540000e+02,65.447559,254.000000,...,4.000000,1.000000,2.000000,1.000000,1.000000,2.000000,0.000000,0.000000,2.000000,3.000000
75%,0.937146,111.00000,2.000000,3.000000,16.000000,18.000000,2.158000e+03,1.824000e+03,3289.241089,254.000000,...,7.000000,1.000000,3.000000,2.000000,1.000000,4.000000,0.000000,0.000000,4.000000,6.000000
max,59.999989,130.00000,12.000000,6.000000,10646.000000,11018.000000,1.435577e+07,1.465753e+07,1000000.000000,255.000000,...,63.000000,6.000000,59.000000,59.000000,38.000000,63.000000,2.000000,16.000000,60.000000,62.000000


Preprocessing done by Laurens D'Hooge

## Creating the model
- Build an agent to classify network flow automatically
- Feed a packet that gets classified
- Want the classification to be equal to the label

In [5]:
def balancing_dataset_sampler_df(df, train_frac=0.2, val_frac=0.1, test_frac=0.7):
    col = df.columns[-1]
    print(col)
    cols = df.columns[:-1]
    print(cols)
    vc = df[col].value_counts()
    print(vc)
    n = vc.iloc[-1]
    print(n)
    m = vc.iloc[0]
    print(m)
    print(int(m-n))
    initial_cut = df.loc[df[col] == vc.index[0]].sample(n=int(m-n), replace=False)
    print(initial_cut.index)
    df = df.drop(index=initial_cut.index)
    vc = df[col].value_counts()
    print(vc)
    print(int(n*train_frac))
    train_df = df.groupby(col).apply(lambda x: x.sample(n=int(n*train_frac), replace=False))
    train_df.reset_index(level=0, inplace=True, drop=True)
    df = df.drop(index=train_df.index)

    validation_df = df.groupby(col).apply(lambda x: x.sample(n=int(n*val_frac), replace=False))
    validation_df.reset_index(level=0, inplace=True, drop=True)
    df = df.drop(index=validation_df.index)

    test_df = df.groupby(col).apply(lambda x: x.sample(n=int(n*test_frac), replace=False))
    test_df.reset_index(level=0, inplace=True, drop=True)
    df = df.drop(index=test_df.index)

    return train_df[cols], train_df[col], validation_df[cols], validation_df[col], test_df[cols], test_df[col]


In [6]:
def nslkdd_split_df(train_df, test_df):
    train_col = train_df.columns[-1]
    print(train_col)
    train_cols = train_df.columns[:-1]
    print(train_cols)
    test_col = test_df.columns[-1]
    print(test_col)
    test_cols = test_df.columns[:-1]
    print(test_cols)

    return train_df[train_cols], train_df[train_col], test_df[test_cols], test_df[test_col]

In [7]:
print(nslkdd_train['attack_cat'].value_counts())
nslkdd_train.drop('label', inplace=True, axis=1)
nslkdd_train['attack_cat'] = nslkdd_train['attack_cat'].astype('object')
atk_idx = nslkdd_train.loc[nslkdd_train['attack_cat'] != "normal"].index
nslkdd_train.loc[atk_idx, 'attack_cat'] = 1.0
nslkdd_train.loc[nslkdd_train.index.difference(atk_idx), 'attack_cat'] = 0.0
nslkdd_train['attack_cat'] = nslkdd_train['attack_cat'].astype(dtype=np.float32)

normal            34200
exploits           7126
fuzzers            4394
generic            3640
reconnaissance     2469
dos                1315
shellcode           377
analysis            312
backdoor             70
worms                43
Name: attack_cat, dtype: int64


In [8]:
print(nslkdd_test['attack_cat'].value_counts())
nslkdd_test.drop('label', inplace=True, axis=1)
nslkdd_test['attack_cat'] = nslkdd_test['attack_cat'].astype('object')
atk_idx = nslkdd_test.loc[nslkdd_test['attack_cat'] != "normal"].index
nslkdd_test.loc[atk_idx, 'attack_cat'] = 1.0
nslkdd_test.loc[nslkdd_test.index.difference(atk_idx), 'attack_cat'] = 0.0
nslkdd_test['attack_cat'] = nslkdd_test['attack_cat'].astype(dtype=np.float32)

normal            51815
exploits          18721
fuzzers           14889
reconnaissance     6422
generic            3924
dos                3004
shellcode          1089
analysis            542
backdoor            513
worms               121
Name: attack_cat, dtype: int64


In [9]:
x_train, y_train, x_test, y_test  = nslkdd_split_df(nslkdd_train, nslkdd_test)

attack_cat
Index(['dur', 'proto', 'service', 'state', 'spkts', 'dpkts', 'sbytes',
       'dbytes', 'rate', 'sttl', 'dttl', 'sload', 'dload', 'sloss', 'dloss',
       'sinpkt', 'dinpkt', 'sjit', 'djit', 'swin', 'stcpb', 'dtcpb', 'dwin',
       'tcprtt', 'synack', 'ackdat', 'smean', 'dmean', 'trans_depth',
       'response_body_len', 'ct_srv_src', 'ct_state_ttl', 'ct_dst_ltm',
       'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm',
       'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd', 'ct_src_ltm',
       'ct_srv_dst', 'is_sm_ips_ports'],
      dtype='object')
attack_cat
Index(['dur', 'proto', 'service', 'state', 'spkts', 'dpkts', 'sbytes',
       'dbytes', 'rate', 'sttl', 'dttl', 'sload', 'dload', 'sloss', 'dloss',
       'sinpkt', 'dinpkt', 'sjit', 'djit', 'swin', 'stcpb', 'dtcpb', 'dwin',
       'tcprtt', 'synack', 'ackdat', 'smean', 'dmean', 'trans_depth',
       'response_body_len', 'ct_srv_src', 'ct_state_ttl', 'ct_dst_ltm',
       'ct_src_dport_ltm', 'ct_dst_sport_ltm'

In [10]:
y_train = y_train.to_frame()
y_test = y_test.to_frame()

# custom keys -> replace by index

x_train = x_train.set_index([pd.Index(range (0, len(x_train)))])
y_train = y_train.set_index([pd.Index(range (0, len(y_train)))])
x_test = x_test.set_index([pd.Index(range (0, len(x_test)))])
y_test = y_test.set_index([pd.Index(range (0, len(y_test)))])


### Custom environment

In [11]:
class IdsEnv(gym.Env):
    def __init__(self, images_per_episode=1, dataset=(x_train, y_train), random=True):
        # Actions we can take, classify as malicious or non-malicious (later also the correct attack)
        # change to 19 if detectiong all different attacks
        self.action_space = gym.spaces.Discrete(2)
         # All the features we have, len(important_features) - 1 features and 1 label. Label should not be included
        self.observation_space = gym.spaces.Box(low=float('-inf'), high=float('inf'), shape=(len(nslkdd_train.columns) - 1,))
        self.images_per_episode = images_per_episode
        self.step_count = 0

        self.x, self.y = dataset
        self.random = random
        self.dataset_idx = 0
    
    def step(self, action):
        done = False
        reward = int(action == self.expected_action)
        current_label = self.expected_action
        obs = self._next_obs()

        self.step_count += 1
        if self.step_count >= self.images_per_episode:
            done = True

        return obs, reward, done, {'label': current_label}

    def _next_obs(self):
        if self.random:
            next_obs_idx = random.randint(0, len(self.x) - 1)
            self.expected_action = int(self.y.iloc[next_obs_idx,:])
            obs = self.x.iloc[next_obs_idx,:]

        else:
            obs = self.x.iloc[self.dataset_idx]
            self.expected_action = int(self.y.iloc[self.dataset_idx])

            self.dataset_idx += 1
            if self.dataset_idx >= len(self.x):
                raise StopIteration()
        return obs
    
    def reset(self):
        self.step_count = 0

        obs = self._next_obs()
        return obs

Train classifier using dqn

In [12]:
def ids_ddqn():
    env = IdsEnv(images_per_episode=1)
    env = bench.Monitor(env, logger.get_dir())

    model = deepq.DQN(
        MlpPolicy, 
        env, 
        policy_kwargs=dict(dueling=True),
        double_q=True,
        verbose=1, 
        learning_rate=0.00025,
        buffer_size=1000000,
        exploration_fraction=0.1,
        exploration_final_eps=0.1,
        train_freq=4,
        learning_starts=50000,
        target_network_update_freq=10000,
        gamma=0.99,
        param_noise=True,
        prioritized_replay=True,
        prioritized_replay_alpha=0.6,
        batch_size=32,
    )
    model.learn(
        total_timesteps=int(1.0e5),
        log_interval=int(1.0e5),
        
    )

    env.close()
    
    return model

start_time = time.time()
ddqn_model_1 = ids_ddqn()
print()
print("DQN 1 Training Time:", time.time() - start_time)
start_time = time.time()
ddqn_model_2 = ids_ddqn()
print()
print("DQN 2 Training Time:", time.time() - start_time)
start_time = time.time()
ddqn_model_3 = ids_ddqn()
print()
print("DQN 3 Training Time:", time.time() - start_time)
start_time = time.time()
ddqn_model_4 = ids_ddqn()
print()
print("DQN 4 Training Time:", time.time() - start_time)








Instructions for updating:
Use keras.layers.flatten instead.


/opt/conda/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


--------------------------------------
| % time spent exploring  | 9        |
| episodes                | 100000   |
| mean 100 episode reward | 0.4      |
| steps                   | 99999    |
--------------------------------------

DQN 1 Training Time: 893.9317739009857


/opt/conda/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


--------------------------------------
| % time spent exploring  | 9        |
| episodes                | 100000   |
| mean 100 episode reward | 0.5      |
| steps                   | 99999    |
--------------------------------------

DQN 2 Training Time: 894.2930254936218


/opt/conda/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


--------------------------------------
| % time spent exploring  | 9        |
| episodes                | 100000   |
| mean 100 episode reward | 0.7      |
| steps                   | 99999    |
--------------------------------------

DQN 3 Training Time: 891.1198127269745


/opt/conda/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


--------------------------------------
| % time spent exploring  | 9        |
| episodes                | 100000   |
| mean 100 episode reward | 0.6      |
| steps                   | 99999    |
--------------------------------------

DQN 4 Training Time: 862.8160674571991


In [13]:
# 0 is benign (positive), 1 is malicious (negative) 
def ids_eval(model):
    TP, FP, TN, FN = 0,0,0,0
    env = IdsEnv(images_per_episode=1, dataset=(x_test, y_test), random=False)
    obs, done = env.reset(), False
    try:
        while True:
            obs, done = env.reset(), False
            while not done:
                obs, rew, done, info = env.step(model.predict(obs)[0])
                label = info['label']
                if label == 0 and rew > 0:
                    TP += 1
                if label == 0 and rew == 0:
                    FP += 1
                if label == 1 and rew > 0:
                    TN += 1
                if label == 1 and rew == 0:
                    FN += 1

    except StopIteration:
        accuracy = (float(TP + TN) / (TP + FP + FN + TN)) 
        precision = (float(TP) / (TP + FP))
        try:
            recall = (float(TP) / (TP + FN)) # = TPR = Sensitivity
        except:
            recall = 0
        try:
            FPR = (float(FP) / (TN + FP)) # 1 - specificity
        except:
            FPR = 0
        try:
            f1_score = 2 * (precision * recall) / (precision + recall)
        except:
            f1_score = 0
        print()
        print('validation done...')
        print('Accuracy: {0}%'.format(accuracy * 100))
        print('Precision: {0}%'.format(precision * 100))
        print('Recall/TPR/Sensitivity: {0}%'.format(recall * 100))
        print('FPR: {0}%'.format(FPR * 100))
        print('F1 score: {0}'.format(f1_score))
    return [accuracy, precision, recall, FPR, f1_score]

models = [ddqn_model_1, ddqn_model_2, ddqn_model_3, ddqn_model_4]

results_1 = ids_eval(ddqn_model_1)
results_2 = ids_eval(ddqn_model_2)
results_3 = ids_eval(ddqn_model_3)
results_4 = ids_eval(ddqn_model_4)
total_results = [-1,-1,-1,-1,-1]
accuracies = [results_1[0], results_2[0], results_3[0], results_4[0]]

for i in range(len(results_1)):
    total_results[i] = (results_1[i] + results_2[i] + results_3[i] + results_4[i] )/ 4

print()    
print('Total validation done...')
print('Accuracy: {0}%'.format(total_results[0] * 100))
print('Precision: {0}%'.format(total_results[1] * 100))
print('Recall/TPR/Sensitivity: {0}%'.format(total_results[2] * 100))
print('FPR: {0}%'.format(total_results[3] * 100))
print('F1 score: {0}'.format(total_results[4]))

highest = 0
for i in range(4):
    if accuracies[i] > accuracies[highest]:
        highest = i

print()
print("Saving model {0}".format(highest + 1))
models[highest].save('pn_per_dddqn_cicddos2019_allfeat.pkl')



/opt/conda/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))



validation done...
Accuracy: 48.77174924285912%
Precision: 0.015454756201220925%
Recall/TPR/Sensitivity: 66.66666666666666%
FPR: 51.23037633876428%
F1 score: 0.0003090234857849197

validation done...
Accuracy: 46.76062471545359%
Precision: 52.333668186384365%
Recall/TPR/Sensitivity: 48.19598633646456%
FPR: 55.03903635958064%
F1 score: 0.5017967621235135

validation done...
Accuracy: 59.42912567548843%
Precision: 95.37902789583495%
Recall/TPR/Sensitivity: 56.12240258263993%
FPR: 18.307056482473598%
F1 score: 0.7066468197171809

validation done...
Accuracy: 50.525544844513945%
Precision: 98.12224712155165%
Recall/TPR/Sensitivity: 50.88970823982045%
FPR: 79.02439024390245%
F1 score: 0.6702029398569657

Total validation done...
Accuracy: 51.37176111957877%
Precision: 61.46259948999304%
Recall/TPR/Sensitivity: 55.46869095639791%
FPR: 50.900214856180234%
F1 score: 0.46973888629586125

Saving model 3
